### 출력 파서

LLM이 생성한 출력 결과를 필요에 맞게 가공, 구조화

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1, # 창의성 (0.0~2.0)
    google_api_key=gemini_api_key
)

In [3]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()


In [4]:
output_parser

StrOutputParser()

In [5]:
from langchain_core.prompts import PromptTemplate

template = "{language} 할 수 있어?"

prompt = PromptTemplate.from_template(template)

In [6]:
chain = prompt | llm | output_parser

In [7]:
chain.invoke({"language": "한국어"})

'네, 한국어 할 수 있습니다. 무엇을 도와드릴까요?'

### 영어 회화 예제

In [8]:
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 
주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고해 주세요.

# 상황:
{question}

# FORMAT:
- 영어 회화:
- 한글 해석:
"""

In [13]:
prompt = PromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1, # 창의성 (0.0~2.0)
    google_api_key=gemini_api_key
)

chain = prompt | llm | output_parser

In [14]:
result = chain.invoke({"question": "저는 식당에서 음식을 주문하고 싶어요."})

In [11]:
result

"안녕하세요! 식당에서 주문하는 상황, 아주 쉽고 친절하게 알려드릴게요. 😊\n\n---\n\n**상황:** 저는 식당에서 음식을 주문하고 싶어요.\n\n**[FORMAT]**\n\n- 영어 회화: Excuse me.\n- 한글 해석: 저기요. / 실례합니다.\n(✨ **선생님 팁:** 직원을 부를 때 가장 흔하고 공손한 표현이에요!)\n\n- 영어 회화: Hello! Are you ready to order?\n- 한글 해석: 안녕하세요! 주문하시겠어요?\n(✨ **선생님 팁:** 직원이 '주문할 준비가 되셨나요?' 하고 묻는 표현이에요.)\n\n- 영어 회화: Yes, I'd like the [menu item], please.\n- 한글 해석: 네, [메뉴 이름]으로 주세요.\n(✨ **선생님 팁:** 가장 많이 쓰는 주문 표현이에요. [menu item] 자리에 원하는 메뉴 이름을 넣으면 돼요. 예를 들어, 'I'd like the **pasta**, please.' (파스타로 주세요.))\n\n- 영어 회화: Anything else?\n- 한글 해석: 다른 건 없으세요? / 더 필요하신 건 없으세요?\n(✨ **선생님 팁:** 추가 주문이 있는지 물어볼 때 쓰는 말이에요.)\n\n- 영어 회화: No, that's all, thank you.\n- 한글 해석: 아니요, 그게 다예요. 감사합니다.\n(✨ **선생님 팁:** 주문이 끝났을 때 이렇게 말하면 돼요.)\n\n---\n\n어때요? 참 쉽죠? 이 표현들만 알아도 식당에서 자신 있게 주문할 수 있을 거예요! 👍"

In [15]:
print(result)

안녕하세요! 식당에서 음식을 주문하는 상황이군요. 아주 쉽고 친절하게 알려드릴게요! 😊

식당에서 주문할 때 가장 중요한 표현 몇 가지를 먼저 알아볼까요?

*   **"I'd like to order, please."** (주문하고 싶어요.)
*   **"What do you recommend?"** (뭐 추천하세요?)
*   **"I'll have the [음식 이름], please."** (저는 [음식 이름]으로 할게요.)
*   **"And a [음료 이름], please."** (그리고 [음료 이름] 주세요.)

자, 그럼 이 표현들을 사용해서 대화를 만들어 볼까요?

---

# FORMAT:
-   **영어 회화:**
    **You (나):** Excuse me.
    **Server (직원):** Hello! Are you ready to order?
    **You (나):** Yes, I'd like to order, please. What do you recommend?
    **Server (직원):** Our pasta is very popular, and the chicken salad is also great!
    **You (나):** Hmm, okay. I'll have the pasta, please.
    **Server (직원):** Great choice! And for drinks?
    **You (나):** I'll have a glass of water, please.
    **Server (직원):** Alright. So, one pasta and one water. Is that all?
    **You (나):** Yes, that's all. Thank you!
    **Server (직원):** You're welcome!

-   **한글 해석:**
    **You (나):** 저기요.
    **Server (직원):** 안녕하세요! 주문하시겠어요?
    **You (나):**

### JsonOutputParser

In [16]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

In [23]:
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 
주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고해 주세요.

# 상황:
{question}

# FORMAT:
{format_instructions}
"""

In [24]:
parser.get_format_instructions()

'Return a JSON object.'

In [25]:
prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions() # LLM에게 JSON 출력 형식을 정확히 지시하는 문자열 반환
    }
)

In [26]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1, # 창의성 (0.0~2.0)
    google_api_key=gemini_api_key
)

chain = prompt | llm | parser

In [27]:
user_question = "친구와 저녁 메뉴를 정하는 상황"
response = chain.invoke({"question": user_question})

In [28]:
response

{'title': 'Deciding on Dinner (저녁 메뉴 정하기)',
 'description': '친구와 저녁 메뉴를 정하는 상황',
 'conversation': [{'speaker': 'Sarah',
   'line': 'Hey Tom, what do you want for dinner tonight?'},
  {'speaker': 'Tom',
   'line': "Hmm, good question. I'm pretty hungry. Do you have anything in mind?"},
  {'speaker': 'Sarah',
   'line': "Not really. I'm open to suggestions. How about Italian? Pasta or pizza?"},
  {'speaker': 'Tom',
   'line': "Italian sounds good, but we had pizza last week, didn't we?"},
  {'speaker': 'Sarah',
   'line': 'Oh, right! My memory. Okay, then maybe something different. What about Korean BBQ?'},
  {'speaker': 'Tom',
   'line': "Ooh, Korean BBQ! That's a great idea! I haven't had that in a while."},
  {'speaker': 'Sarah',
   'line': "Me neither! There's that new place downtown, 'Seoul Grill.' I heard it's really good."},
  {'speaker': 'Tom', 'line': 'Oh yeah, I saw that! Is it expensive?'},
  {'speaker': 'Sarah',
   'line': "I don't think so. It looked pretty reasonable online

### JsonOutputParser + Pydantic

In [37]:
from pydantic import BaseModel, Field

class EnglishConverstion(BaseModel):
    title: str = Field(description="상황에 대한 제목")
    dialogue: list = Field(description="상황에 맞는 영어 대화 리스트")
    explanation: str = Field(description="대화에 대한 쉬운 설명")

In [38]:
parser = JsonOutputParser(pydantic_object=EnglishConverstion)

In [39]:
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 
주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고해 주세요.
모든 키 이름은 정확히 이 예시와 일치해야 합니다.

# 상황:
{question}

# FORMAT:
{format_instructions}

# 예시:
{{
    "title": "저녁 식사 정하기 (Deciding Dinner)",
    "dialogue": [
        {{"speaker": "A", "text": "What?"}},
        {{"speaker": "B", "text": "pizza."}},
    ],
    "explanation": "친구와 저녁 메뉴를 정하는 상황입니다."
}}
"""

In [40]:
prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

In [41]:
chain = prompt | llm | parser

In [42]:
result = chain.invoke({"question": "식당에서 주문하는 상황"})

In [43]:
result

{'title': '식당에서 주문하기 (Ordering at a Restaurant)',
 'dialogue': [{'speaker': 'Server',
   'text': 'Hi there! Welcome to our restaurant. Are you ready to order?'},
  {'speaker': 'Customer', 'text': 'Yes, I think so. What do you recommend?'},
  {'speaker': 'Server',
   'text': 'Our pasta dishes are very popular, especially the Creamy Mushroom Pasta.'},
  {'speaker': 'Customer', 'text': 'Oh, that sounds good. Is it spicy?'},
  {'speaker': 'Server',
   'text': "No, it's not spicy at all. It's very creamy and mild."},
  {'speaker': 'Customer',
   'text': "Okay, I'll have the Creamy Mushroom Pasta then. And a glass of orange juice, please."},
  {'speaker': 'Server',
   'text': 'Alright, so one Creamy Mushroom Pasta and one orange juice. Anything else for you today?'},
  {'speaker': 'Customer', 'text': "No, that's all, thank you."},
  {'speaker': 'Server', 'text': 'Great! Your food will be out shortly.'}],
 'explanation': "손님이 식당에서 메뉴를 고르고 주문하는 상황입니다. 추천 메뉴를 묻고, 음식에 대해 질문한 후 주문을 합니다. 'I'll hav

In [47]:
import pprint

pprint.pprint(result, indent=2)

{ 'dialogue': [ { 'speaker': 'Server',
                  'text': 'Hi there! Welcome to our restaurant. Are you ready '
                          'to order?'},
                { 'speaker': 'Customer',
                  'text': 'Yes, I think so. What do you recommend?'},
                { 'speaker': 'Server',
                  'text': 'Our pasta dishes are very popular, especially the '
                          'Creamy Mushroom Pasta.'},
                { 'speaker': 'Customer',
                  'text': 'Oh, that sounds good. Is it spicy?'},
                { 'speaker': 'Server',
                  'text': "No, it's not spicy at all. It's very creamy and "
                          'mild.'},
                { 'speaker': 'Customer',
                  'text': "Okay, I'll have the Creamy Mushroom Pasta then. And "
                          'a glass of orange juice, please.'},
                { 'speaker': 'Server',
                  'text': 'Alright, so one Creamy Mushroom Pasta and one '
  

In [48]:
type(result)

dict

In [51]:
import json

json_result = json.dumps(result, ensure_ascii=False, indent=2)

In [52]:
type(json_result)

str

In [54]:
print(json_result)

{
  "title": "식당에서 주문하기 (Ordering at a Restaurant)",
  "dialogue": [
    {
      "speaker": "Server",
      "text": "Hi there! Welcome to our restaurant. Are you ready to order?"
    },
    {
      "speaker": "Customer",
      "text": "Yes, I think so. What do you recommend?"
    },
    {
      "speaker": "Server",
      "text": "Our pasta dishes are very popular, especially the Creamy Mushroom Pasta."
    },
    {
      "speaker": "Customer",
      "text": "Oh, that sounds good. Is it spicy?"
    },
    {
      "speaker": "Server",
      "text": "No, it's not spicy at all. It's very creamy and mild."
    },
    {
      "speaker": "Customer",
      "text": "Okay, I'll have the Creamy Mushroom Pasta then. And a glass of orange juice, please."
    },
    {
      "speaker": "Server",
      "text": "Alright, so one Creamy Mushroom Pasta and one orange juice. Anything else for you today?"
    },
    {
      "speaker": "Customer",
      "text": "No, that's all, thank you."
    },
    {
     